In [1]:
import hashlib
import datetime
from collections import defaultdict 

In [2]:
class Transaction:
    def __init__(self, sender, recipient, amount, data=None):
        self.sender = sender
        self.recipient = recipient
        self.amount = amount
        self.data = data
    def __repr__(self):
        return f"Transaction(sender={self.sender}, recipient={self.recipient}, amount={self.amount}, data={self.data})"

In [3]:
def hash_data(data):
    return hashlib.sha256(data.encode()).hexdigest()


class MerkleNode:
    def __init__(self, left=None, right=None, data=None):
        self.left = left 
        self.right = right
        self.hash = self.calculate_hash(data)
    def calculate_hash(self, data):
        if data is not None:
            return hash_data(data)
        else:
            return hash_data(self.left.hash + self.right.hash if self.left and self.right else '')


class MerkleTree:
    def __init__(self):
        self.leaves = []
        self.root = None 
        
    def add(self, data):
        new_node = MerkleNode(data=data)
        self.leaves.append(new_node)
        self.recalculate_tree()

    def recalculate_tree(self):
        nodes = self.leaves[:]
        while len(nodes) > 1:
            if len(nodes) % 2 == 1:
                nodes.append(MerkleNode(data=nodes[-1].hash))
            new_level = []
            for i in range(0, len(nodes), 2):
                new_level.append(MerkleNode(left=nodes[i], right=nodes[i + 1]))
            nodes = new_level
        self.root = nodes[0] if nodes else None

    def root_hash(self):
        return self.root.hash if self.root else ''

In [4]:
class Block:
    def __init__(self, transactions, previous_hash=''):
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = 0
        self.merkle_tree = MerkleTree()
        for transaction in transactions:
            self.merkle_tree.add(transaction.__repr__())
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = f"{self.previous_hash}{self.nonce}{self.merkle_tree.root_hash()}"
        return hash_data(block_string)

    def proof_of_work(self, difficulty):
        target = '0' * difficulty
        while self.hash[:difficulty] != target:
            self.nonce += 1
            self.hash = self.calculate_hash()
        print(f"Block with PoW: {self.hash}")

In [5]:
class Blockchain:
    def __init__(self, difficulty=4):
        self.chain = [] 
        self.difficulty = difficulty 
        self.pending_transactions = [] 
        self.blockchain_merkle_tree = MerkleTree()
        self.create_genesis_block()

    def create_genesis_block(self):
        initial_transactions = [
            Transaction(sender="root", recipient="Alice", amount=1000, data="Initial balance"),
            Transaction(sender="root", recipient="Bob", amount=500, data="Initial balance")
        ]
        genesis_block = Block(transactions=initial_transactions, previous_hash="0")
        genesis_block.proof_of_work(self.difficulty)
        self.chain.append(genesis_block)

    def add_block(self, block):
        block.previous_hash = self.get_latest_block().hash
        block.proof_of_work(self.difficulty)
        self.chain.append(block)
        self.blockchain_merkle_tree.add(block.hash)

    def get_latest_block(self):
        return self.chain[-1]

    def calculate_blockchain_root_hash(self):
        temp_tree = MerkleTree()
        for block in self.chain:
            temp_tree.add(block.hash)
        return temp_tree.root_hash()

    def is_chain_valid(self):
        if self.blockchain_merkle_tree.root_hash() != self.calculate_blockchain_root_hash():
            return False

        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i - 1]

            if current.hash != current.calculate_hash():
                return False

            if current.previous_hash != previous.hash:
                return False

        return True

    def update_balances(self, user_balances, sender, recipient, amount):
        if sender != "root":
            user_balances[sender]['current'] -= amount
            user_balances[sender]['min'] = min(user_balances[sender]['min'], user_balances[sender]['current'])
            user_balances[sender]['max'] = max(user_balances[sender]['max'], user_balances[sender]['current'])
        user_balances[recipient]['current'] += amount
        user_balances[recipient]['min'] = min(user_balances[recipient]['min'], user_balances[recipient]['current'])
        user_balances[recipient]['max'] = max(user_balances[recipient]['max'], user_balances[recipient]['current'])

    def calculate_balances_until(self, block_number):
        user_balances = defaultdict(lambda: {'min': float('inf'), 'max': float('-inf'), 'current': 0})
        for i in range(min(block_number + 1, len(self.chain))):
            for tx in self.chain[i].transactions:
                self.update_balances(user_balances, tx.sender, tx.recipient, tx.amount)
        return user_balances

    def add_transaction(self, transaction):
        if self.calculate_balance(transaction.sender) >= transaction.amount:
            self.pending_transactions.append(transaction)
            print(f"Transaction from {transaction.sender} to {transaction.recipient} for {transaction.amount} added.")
        else:
            print(f"Error: Not enough balance for the transaction from {transaction.sender}.")

    def calculate_balance(self, sender):
        balance = 0
        for block in self.chain:
            for tx in block.transactions:
                if tx.sender == sender:
                    balance -= tx.amount
                if tx.recipient == sender:
                    balance += tx.amount
        for tx in self.pending_transactions:
            if tx.sender == sender:
                balance -= tx.amount
            if tx.recipient == sender:
                balance += tx.amount
        return balance

In [6]:
import json

def save_blockchain(blockchain, filename='blockchain.json'):
    with open(filename, 'w') as file:
        chain_data = []
        for block in blockchain.chain:
            block_info = {
                'transactions': [
                    {
                        'sender': tx.sender,
                        'recipient': tx.recipient,
                        'amount': tx.amount,
                        'data': tx.data
                    } for tx in block.transactions
                ],
                'previous_hash': block.previous_hash,
                'nonce': block.nonce,
                'hash': block.hash
            }
            chain_data.append(block_info)
        json.dump(chain_data, file, indent=4)
    print("Blockchain saved to file.")


def load_blockchain(filename='blockchain.json'):
    with open(filename, 'r') as file:
        chain_data = json.load(file)
        blockchain = Blockchain()
        blockchain.chain = []

        for block_data in chain_data:
            transactions = [
                Transaction(tx['sender'], tx['recipient'], tx['amount'], tx.get('data', None))
                for tx in block_data['transactions']
            ]
            block = Block(transactions, block_data['previous_hash'])
            block.nonce = block_data['nonce']
            block.hash = block.calculate_hash()
            blockchain.chain.append(block)
        print("Blockchain loaded from file with {} blocks.".format(len(blockchain.chain)))

        return blockchain

In [7]:

blockchain = Blockchain()
current_time = datetime.datetime.now(tz=datetime.UTC)
str_current_time = current_time.strftime("%Y-%m-%d %H:%M:%S %Z")

transactions = [
        Transaction(sender="Alice", recipient="Bob", amount=15, data=str_current_time),
        Transaction(sender="Bob", recipient="Charlie", amount=10, data=str_current_time),
        Transaction(sender="Alice", recipient="Charlie", amount=20, data=str_current_time),
        Transaction(sender="Bob", recipient="Alice", amount=50, data=str_current_time),
        Transaction(sender="Bob", recipient="Alice", amount=100, data=str_current_time),
        Transaction(sender="Charlie", recipient="Alice", amount=60, data=str_current_time),
        Transaction(sender="Alice", recipient="Bob", amount=10, data=str_current_time),
        Transaction(sender="Charlie", recipient="Bob", amount=20, data=str_current_time)
]

print("\nAdding transactions to the pending transactions list...")
for tx in transactions:
    blockchain.add_transaction(tx)

transactions_per_block = 3
print("\nCreating blocks with a fixed number of transactions...")
for i in range(0, len(blockchain.pending_transactions), transactions_per_block):
    end_index = i + transactions_per_block
    block_transactions = blockchain.pending_transactions[i:end_index]
    if block_transactions:
        new_block = Block(transactions=block_transactions)
        blockchain.add_block(new_block)

save_blockchain(blockchain, 'blockchain.json')

loaded_blockchain = load_blockchain('blockchain.json')

block_number = 2
print(f"\nAnalyzing loaded block {block_number}...")
if block_number < len(loaded_blockchain.chain):
    specific_block = loaded_blockchain.chain[block_number]
    print(f"Loaded Block {block_number} with hash: {specific_block.hash}")

    print("\nCalculating and displaying balances for the specified block...")
    balances = loaded_blockchain.calculate_balances_until(block_number)
    for user, info in balances.items():
        print(f"User: {user}, Current Balance: {info['current']}, Min: {info['min']}, Max: {info['max']}")
else:
    print("Invalid block number")


Block with PoW: 0000902349191821e513b07f9deafad3ec4816c9d57147f02d450b306a8f209b

Adding transactions to the pending transactions list...
Transaction from Alice to Bob for 15 added.
Transaction from Bob to Charlie for 10 added.
Transaction from Alice to Charlie for 20 added.
Transaction from Bob to Alice for 50 added.
Transaction from Bob to Alice for 100 added.
Error: Not enough balance for the transaction from Charlie.
Transaction from Alice to Bob for 10 added.
Transaction from Charlie to Bob for 20 added.

Creating blocks with a fixed number of transactions...
Block with PoW: 0000ad00c927f4e78e015824f4b7a7340b45d2ca918bc1942f1229cbdcdb1a5a
Block with PoW: 0000f103db26c3273df4149cb9745e3a78326b6361e9e0acb4924faaaf7da123
Block with PoW: 000069aac28a6e859ae2306b58e8a40d2519e6ce4c3e43905235e82fd5172f1e
Blockchain saved to file.
Block with PoW: 0000902349191821e513b07f9deafad3ec4816c9d57147f02d450b306a8f209b
Blockchain loaded from file with 4 blocks.

Analyzing loaded block 2...
Loaded 